In [1]:
import numpy as np
import pandas as pd

### Problem Set 3
Your department at the Telecom. company is
on fire
! Since they hired you, they have boosted
their productivity, and are now capable of taking data-driven decision much more quickly. Given the
development of the new techniques, your boss has decided he wants to patent the data processing
methods that you use, in order to add some value to the company. While this is great news... it
means more work for you ! It is the first time the company looks into this kind of patents, so they
know very little. You have been asked to take all the patents from 2016 in the relevant class, and
study your competitors. You are specially interested in knowing : who is the central player in this
technology area ? Who is the leader ? Who is an irrelevant actor ?
The lawyers will subsequently use this information in order to study their patent portfolios and
make the best of our resources 

In [2]:
# use the data from Google Patents Datasets. You have to register to the free-
# tier of Google BigQuery, and use the patents-public-data/publications table. Identify and extract
# all patents from 2016 from that have a patent classification (cpc) starting with G06Q30/06... Then,
# extract all the patents from 2016-2020 that cite these patents, and build a directed network of
# patent citations

In [3]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/hienle/DSIP-As3-25e0a13c65b2.json"

In [4]:
import google.auth
from google.cloud import bigquery
from google.cloud import bigquery_storage_v1beta1

# Explicitly create a credentials object. This allows you to use the same
# credentials for both the BigQuery and BigQuery Storage clients, avoiding
# unnecessary API calls to fetch duplicate authentication tokens.
credentials, your_project_id = google.auth.default(
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

In [5]:

# Make clients.
bqclient = bigquery.Client(
    credentials=credentials,
    project=your_project_id,
)
bqstorageclient = bigquery_storage_v1beta1.BigQueryStorageClient(
    credentials=credentials
)

In [19]:
import arrow
arrow.get("2016-01", tzinfo="utc")

<Arrow [2016-01-01T00:00:00+00:00]>

In [12]:
query_string = f"""
SELECT ANY_VALUE(a.name) AS assignee_name, ANY_VALUE(pubs.publication_number) as pub_number,
ANY_VALUE(SUBSTR(cpcs.code, 0, 9)) AS cpc_code,
ANY_VALUE(pubs.filing_date) AS filing_date, CAST(FLOOR(ANY_VALUE(filing_date) / 10000) AS INT64) AS filing_year,
ANY_VALUE(pubs.country_code) AS country_code
FROM `patents-public-data.patents.publications` AS pubs, UNNEST(assignee_harmonized) as a,
UNNEST(pubs.cpc) as cpcs
WHERE cpcs.first = TRUE AND REGEXP_CONTAINS(cpcs.code, "G06Q30/06") AND 
filing_date > 20160101
GROUP BY application_number
"""

dataframe = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
)
dataframe.head()

,assignee_name,pub_number,cpc_code,filing_date,filing_year,country_code
0,LIM SOON SEOP,KR-20200021490-A,G06Q30/06,20200210,2020,KR
1,LIM SOON SEOP,KR-20200021488-A,G06Q30/06,20200210,2020,KR
2,ELECTRONIC COMMODITIES EXCHANGE L P,AU-2020200803-A1,G06Q30/06,20200204,2020,AU
3,LIM SOON SEOP,KR-20200021491-A,G06Q30/06,20200210,2020,KR
4,STRIPE INC,AU-2020201049-A1,G06Q30/06,20200213,2020,AU


In [59]:
dataframe.shape

(10686, 6)

In [60]:
dataframe.to_csv("all_patents_G06Q30.csv")

In [13]:
# Then,
# extract all the patents from 2016-2020 that cite these patents, and build a directed network of
# patent citations. Try to answer the questions stated above

# INNER JOIN

query_string2 = """
SELECT cited_assignee, refs.cited_publication_number, citing_publication_number, citing_assignee, cited_cpc_code, 
citing_cpc_code FROM
(SELECT ANY_VALUE(a.name) AS cited_assignee, ANY_VALUE(pubs.publication_number) as 
cited_publication_number,
ANY_VALUE(SUBSTR(cpcs.code, 0, 9)) AS cited_cpc_code, ANY_VALUE(pubs.filing_date) AS filing_date
FROM `patents-public-data.patents.publications` AS pubs, UNNEST(assignee_harmonized) as a,
UNNEST(pubs.cpc) as cpcs
WHERE cpcs.first = TRUE AND REGEXP_CONTAINS(cpcs.code, "G06Q30/06") AND 
filing_date > 20160101
GROUP BY application_number
) as refs
JOIN (
  SELECT pubs.publication_number AS citing_publication_number, cite.publication_number AS cited_publication_number, citing_assignee_s.name AS citing_assignee, SUBSTR(cpcs.code, 0, 9) AS citing_cpc_code, 
  pubs.filing_date AS citing_filing_date
  FROM `patents-public-data.patents.publications` AS pubs, UNNEST(citation) AS cite, UNNEST(assignee_harmonized) AS citing_assignee_s, UNNEST(cpc) AS cpcs
  WHERE cpcs.first = TRUE AND pubs.filing_date > 20160101
) AS pubs
ON refs.cited_publication_number = pubs.cited_publication_number
GROUP BY citing_publication_number, refs.cited_publication_number, pubs.cited_publication_number, citing_assignee, cited_assignee, citing_cpc_code, cited_cpc_code, citing_filing_date
"""

dataframe2 = (
    bqclient.query(query_string2)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
)
dataframe2.head()

,cited_assignee,cited_publication_number,citing_publication_number,citing_assignee,cited_cpc_code,citing_cpc_code
0,GRABANGO CO,US-2018240180-A1,US-2019096209-A1,INTEL CORP,G06Q30/06,G08B13/24
1,EMBL RETAIL INC,US-2018300791-A1,US-10420397-B2,BLACK BRASS INC,G06Q30/06,A43D1/025
2,ONETRUST LLC,US-9691090-B1,US-10574705-B2,ONETRUST LLC,G06Q30/06,H04L63/20
3,NewStore,US-2017011449-A1,US-10134001-B2,THEATRO LABS INC,G06Q30/06,G06Q10/06
4,RARE MILE TECH INC,US-2016217522-A1,US-10417671-B2,YEXT INC,G06Q30/06,G06Q30/02


In [15]:
dataframe2.to_csv("citation_2016.csv")

In [14]:
dataframe2.shape

(4603, 6)